# Imports

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from skimage import io
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from PredictLetter import mp_detection, formatPoints

In [2]:
#### create array to signify the actions for iterating

# actions to be detected by model
actions = np.array(['A', 'B', 'C', 'D', 'del', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'nothing', 'O', 'P', 'Q', 'R', 'S', 'space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

# Preprocess Data and create labels

In [3]:
#### create all the path variables

# path for exported data (numpy arrays)
DATA_PATH = os.path.join('MP_DATA')

# the path for the data used to train our model
TRAIN_PATH = os.path.join(DATA_PATH, 'asl_alphabet_train')

# the path for the data used to test our model
TEST_PATH = os.path.join(DATA_PATH, 'asl_alphabet_test')

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
sequences, labels = [], []

# the number of files we are loading in from our converted numpy dataset to train out model
# numTrainFiles = 4000
for action in actions:
    # count = 0 
    for filename in os.listdir(os.path.join(TRAIN_PATH, action)):
        window = []
        keypoints = np.load(os.path.join(TRAIN_PATH, action, filename))
        window.append(keypoints)
        sequences.append(window)
        labels.append(label_map[action])
        # count += 1
        # if numTrainFiles == count:
            # print(f"Loading Sequences and labels: '{action}' complete")
            # break

    print(f"Loading Sequences and labels: '{action}' complete")

Loading Sequences and labels: 'A' complete
Loading Sequences and labels: 'B' complete
Loading Sequences and labels: 'C' complete
Loading Sequences and labels: 'D' complete
Loading Sequences and labels: 'del' complete
Loading Sequences and labels: 'E' complete
Loading Sequences and labels: 'F' complete
Loading Sequences and labels: 'G' complete
Loading Sequences and labels: 'H' complete
Loading Sequences and labels: 'I' complete
Loading Sequences and labels: 'J' complete
Loading Sequences and labels: 'K' complete
Loading Sequences and labels: 'L' complete
Loading Sequences and labels: 'M' complete
Loading Sequences and labels: 'N' complete
Loading Sequences and labels: 'nothing' complete
Loading Sequences and labels: 'O' complete
Loading Sequences and labels: 'P' complete
Loading Sequences and labels: 'Q' complete
Loading Sequences and labels: 'R' complete
Loading Sequences and labels: 'S' complete
Loading Sequences and labels: 'space' complete
Loading Sequences and labels: 'T' complete

In [7]:
# the number of files we are choosing to train the model at the moment
# this should match our length of sequences
numTrainFiles*(len(actions))

116000

In [7]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [10]:
print(f"This should be {len(sequences)} if numTrainFiles is {numTrainFiles}: {len(y)}")
print(f"This should be 29 always: {len(y[0])}")

This should be 113665 if numTrainFiles is 4000: 113665
This should be 29 always: 29


# Splitting our data into training and testing partitions for our model training

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) 

# Build and Train LSTM Neural Network

In [9]:
# setting up the console logs to track info while training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# creating and building the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(1,126)))
model.add(LSTM(128, return_sequences=True, activation="relu"))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [10]:
# Here we train the model
model.fit(X_train, y_train,epochs=2000, callbacks=[tb_callback])

Epoch 1/2000


10866/10866 [==============================] - 58s 5ms/step - loss: 1.1725 - categorical_accuracy: 0.5935
Epoch 2/2000
10866/10866 [==============================] - 50s 5ms/step - loss: 0.5070 - categorical_accuracy: 0.8270
Epoch 3/2000
10866/10866 [==============================] - 50s 5ms/step - loss: 0.3461 - categorical_accuracy: 0.8872
Epoch 4/2000
10866/10866 [==============================] - 51s 5ms/step - loss: 0.2761 - categorical_accuracy: 0.9110
Epoch 5/2000
10866/10866 [==============================] - 51s 5ms/step - loss: 0.2401 - categorical_accuracy: 0.9231
Epoch 6/2000
10866/10866 [==============================] - 50s 5ms/step - loss: 0.2146 - categorical_accuracy: 0.9316
Epoch 7/2000
10866/10866 [==============================] - 50s 5ms/step - loss: 0.1933 - categorical_accuracy: 0.9394
Epoch 8/2000
10866/10866 [==============================] - 51s 5ms/step - loss: 0.1773 - categorical_accuracy: 0.9439
Epoch 9/2000
10866/10866 [====================

KeyboardInterrupt: 

In [11]:
# Here we save the model to be used later 
model.save("asl_Alphabet_Model_v.0.2.h5")

C:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Evaluation using confusion matrix and accuracy

In [12]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

572/572 [==============================] - 2s 3ms/step


In [14]:
# matrix representation for checking model accuracy
multilabel_confusion_matrix(ytrue,yhat)

array([[[17606,    35],
        [   31,   629]],

       [[17587,    18],
        [    9,   687]],

       [[17677,     8],
        [   16,   600]],

       [[17598,    11],
        [   25,   667]],

       [[17724,     1],
        [   20,   556]],

       [[17562,    43],
        [    6,   690]],

       [[17625,     2],
        [    8,   666]],

       [[17652,    10],
        [   16,   623]],

       [[17792,     7],
        [   13,   489]],

       [[17629,    20],
        [   16,   636]],

       [[17670,     2],
        [   14,   615]],

       [[17676,    16],
        [   15,   594]],

       [[17631,     3],
        [   10,   657]],

       [[17575,    28],
        [   30,   668]],

       [[17579,    26],
        [   31,   665]],

       [[17817,     0],
        [    0,   484]],

       [[17596,    53],
        [    7,   645]],

       [[17681,    33],
        [   17,   570]],

       [[17654,    27],
        [   28,   592]],

       [[17693,    19],
        [   15,   574]],



In [15]:
# percentage accuracy
accuracy_score(ytrue,yhat)

0.9710944757117097